In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

out_ch = 10
in_ch = 100

class double_conv(tf.Module):
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = keras.Sequential([
            layers.Conv2D(out_ch, 3, padding = "SAME", activation=None, input_shape = [28,28, in_ch]), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Conv2D(out_ch, 3, padding = "SAME", activation=None), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu')            
        ])
    def forward(self, x):
        x = self.conv(x)
        return x

class inconv(tf.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)
    def forward(self, x):
        x = self.conv(x)
        return x

class down(tf.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = keras.Sequential(
            layers.MaxPooling2D(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x
        

class up(tf.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = layers.UpSampling2D(size = (2,2), interpolation='bilinear')
        else:
            self.up = layers.Conv2DTranspose(in_ch//2, 2, stride=(2, 2))

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffX = x1.size()[2] - x2.size()[2]
        diffY = x1.size()[3] - x2.size()[3]
        x2 = F.pad(x2, (diffX // 2, int(diffX / 2),
                        diffY // 2, int(diffY / 2)))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x
    
model = up(in_ch, out_ch)

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

out_ch = 10
in_ch = 100
# model = keras.Sequential([
#     layers.Conv2D(out_ch, 3, padding = "SAME", activation=None, input_shape=(28,28,in_ch)), # keras에서 input_channel의 수가 중요한가?
#     layers.BatchNormalization(),
#     layers.Activation('relu'),
#     layers.Conv2D(out_ch, 3, padding = "SAME", activation=None), # keras에서 input_channel의 수가 중요한가?
#     layers.BatchNormalization(),
#     layers.Activation('relu')   
# ])

class double_conv(tf.Module):
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = keras.Sequential([
            layers.Conv2D(out_ch, 3, padding = "SAME", activation=None, input_shape = [in_ch]), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Conv2D(out_ch, 3, padding = "SAME", activation=None), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu')            
        ])
    def forward(self, x):
        x = self.conv(x)
        return x

model = double_conv(in_ch, out_ch)
# model.forward(model)
# print(model.summary())


ValueError: Input 0 of layer conv2d_28 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: (None, 100)

In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("echo", help="write the string u want to echo")
args = parser.parse_args()
print(args.echo)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable

x = torch.arange(10).reshape(5,2)
# print(x)
print(torch.split(x, 2),1)
# print(torch.split(x, [1,4]))

y = np.arange(10).reshape(5,2)
try:
    print(tf.split(y, 2),1)
except:
    print("Hello")
# print(tf.split(y, [1,4]))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable


print(nn.ModuleList(nn.Linear(10,10) for i in range(10)))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.autograd import Variable

out_ch = 10
in_ch = 3

model = keras.Sequential([
            layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None, input_shape = [28,28, in_ch]), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu')            
        ])

# model.summary()
cell_list = []
for i in range(10):
    cell_list.append(model)
for i in range(10):
    print(cell_list[i].summary())

In [ ]:
model1 =  nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
cell_list1 = []
for i in range(10):
    cell_list1.append(model1)
ModuleList1 = nn.ModuleList(cell_list1)
print(ModuleList1[0])

In [ ]:
class double_conv(tf.Module):
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = keras.Sequential([
            layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None, input_shape = [28,28, in_ch]), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu'),
            layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), # keras에서 input_channel의 수가 중요한가?
            layers.BatchNormalization(),
            layers.Activation('relu')            
        ])
    def forward(self, x):
        x = self.conv(x)
        return x

model2 = double_conv(in_ch, out_ch)



In [ ]:
class double_conv1(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv1, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x
    

    
model3 = double_conv1(in_ch, out_ch)

In [ ]:
ts = np.random.randn(28*28*3).reshape(1,28, 28, 3)

print(model2.conv(ts))

In [ ]:
# 관련 라이브러리 import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True
transformation = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST('data/', train = True, transform = transformation, download = True)
test_dataset = datasets.MNIST('data/', train = False, transform = transformation, download = True)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [ ]:
sample_data = next(iter(train_loader))
def plot_img(image):
    image = image.numpy()[0]
    mean = 0.1307
    std = 0.3081
    image = ((std * image) + mean) 
    plt.imshow(image, cmap = 'gray') 
    plt.show()
plot_img(sample_data[0][2])
plot_img(sample_data[0][1])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d(p = 0.1)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1) 
model = Net()
# if is_cuda:
#     model.cuda()
optimizer = optim.SGD(model.parameters(), lr = 0.001)
data, target = next(iter(train_loader))
output = model(Variable(data.cuda()))
print('output:', output.size(), "\ntarget:", target.size())

In [ ]:
import tensorflow as tf
import torch

x = tf.reshape(tf.range(12), (3,4))
print(x)
p, q, r = tf.unstack(x)
p.shape.as_list()
print(p,q,r)

y = torch.reshape(torch.tensor(range(12)), (3,4))

a,b,c = torch.unbind(y)
print(a,b,c)


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F
import operator

vgg16_bn = models.vgg16_bn(pretrained=True).features

In [ ]:
import tensorflow as tf

tf_vgg16 = tf.keras.applications.VGG16(
    include_top=False
)

In [ ]:
# print(vgg16_bn)

In [ ]:
print(tf_vgg16.summary())

In [ ]:
import numpy as np
x = np.arange(9)
x = x.reshape(1,1,3,3)
y = tf_vgg16(x)
y

In [ ]:
x

In [ ]:
import torch
x = torch.zeros(2, 1, 2, 1, 2)
x.size()


In [ ]:
y = torch.squeeze(x)
y.size()


In [ ]:
y = torch.squeeze(x, 0)
y.size()


In [ ]:
y = torch.squeeze(x, 1)
y.size()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
test_image_dir = "C:/Users/Lim-LAB/Desktop/Git/Robust-Lane-Detection/Tensorflow/LaneDetectionCode/data/testset/image/1_1.jpg"
test_label_dir = "C:/Users/Lim-LAB/Desktop/Git/Robust-Lane-Detection/Tensorflow/LaneDetectionCode/data/testset/truth/1_13.jpg"
data = Image.open(test_image_dir)
label = Image.open(test_label_dir)

ndata = np.array(data)
plt.imshow(ndata)
plt.show()
ndata = ndata[np.newaxis, ...]
nlabel = np.array(label)
plt.imshow(nlabel)
nlabel = nlabel[np.newaxis, ... , np.newaxis]
ndata = ndata/255.
# print(ndata)
nlabel = nlabel/255.
print(nlabel.shape)
plt.show()
test_ds = tf.data.Dataset.from_tensor_slices((ndata, nlabel))
print(type(test_ds))

for img, label in test_ds.take(1):
    # print(img.shape, label.shape)
    model(img, img)

In [16]:
import tensorflow as tf
import config
from tensorflow import keras
from tensorflow.keras import layers
from utils_tensorflow import *
import operator
from config import args_setting
import numpy as np

def unpool(value, name='unpool'):
    """N-dimensional version of the unpooling operation from
    https://www.robots.ox.ac.uk/~vgg/rg/papers/Dosovitskiy_Learning_to_Generate_2015_CVPR_paper.pdf

    :param value: A Tensor of shape [b, d0, d1, ..., dn, ch]
    :return: A Tensor of shape [b, 2*d0, 2*d1, ..., 2*dn, ch]
    """
    # print(tf.name_scope)
    with tf.name_scope(name) as scope:
        sh = value.get_shape().as_list()
        print(sh)
        dim = len(sh[1:-1])
        print(dim)
        out = (tf.reshape(value, [-1] + sh[-dim:]))
        # print(out)
        for i in range(dim, 0, -1):
            # print(i)
            out = tf.concat([out, tf.zeros_like(out)], i)
        print(out)
        out_size = [-1] + [s * 2 for s in sh[1:-1]] + [sh[-1]]
        print(out_size)
        out = tf.reshape(out, out_size, name=scope)
    return out

def generate_model(args):

    use_cuda = args.cuda and tf.test.is_gpu_available()
    device = tf.device("cuda" if use_cuda else "cpu")

    assert args.model in [ 'UNet-ConvLSTM', 'SegNet-ConvLSTM', 'UNet', 'SegNet']
    if args.model == 'SegNet-ConvLSTM':
        model = SegNet_ConvLSTM().to(device)
    elif args.model == 'SegNet':
        model = SegNet().to(device)
    elif args.model == 'UNet-ConvLSTM':
        model =UNet_ConvLSTM(config.img_channel, config.class_num).to(device)
    elif args.model == 'UNet':
        model = UNet(config.img_channel, config.class_num).to(device)
    return model


class UNet_ConvLSTM(tf.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet_ConvLSTM, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)
        self.convlstm = ConvLSTM(input_size=(8,16),
                                 input_dim=512,
                                 hidden_dim=[512, 512],
                                 kernel_size=(3,3),
                                 num_layers=2,
                                 batch_first=False,
                                 bias=True,
                                 return_all_layers=False)

    def __call__(self, x):
        x = tf.unstack(x, axis=1)
        data = []
        for item in x:
            x1 = self.inc(item)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)
            x5 = self.down4(x4)
            data.append(tf.expand_dims(x5, axis = 0))
        data = layers.concatenate(data, axis=0)
        lstm, _ = self.convlstm(data)
        test = lstm[0][ -1,:, :, :, :]
        x = self.up1(test, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x, test



class UNet(tf.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)

    def __call__(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x


class SegNet_ConvLSTM(tf.Module):
    def __init__(self):
        super(SegNet_ConvLSTM,self).__init__()
        self.vgg16_bn = tf.keras.applications.VGG16(include_top=False)
        self.relu = layers.ReLU(),
        self.index_MaxPool = layers.MaxPooling2D(pool_size=(2,2), strides=2, data_format = 'channels_last')
        self.index_UnPool = unpool(kernel_size=2, stride=2)
        # net struct
        self.conv1_block = keras.Sequential([self.vgg16_bn.layers[0],  # input layer
                                        self.vgg16_bn.layers[1],  # conv2d(3,64,(3,3))
                                        self.vgg16_bn.layers[2]  # conv2d(3,64,(3,3))
                                        ])
        self.conv2_block = keras.Sequential([self.vgg16_bn.layers[4],
                                        self.vgg16_bn.layers[5]
                                        ])
        self.conv3_block = keras.Sequential([self.vgg16_bn.layers[7],
                                        self.vgg16_bn.layers[8],
                                        self.vgg16_bn.layers[9]
                                        ])
        self.conv4_block = keras.Sequential([self.vgg16_bn.layers[11],
                                        self.vgg16_bn.layers[12],
                                        self.vgg16_bn.layers[13]
                                        ])
        self.conv5_block = keras.Sequential([self.vgg16_bn.layers[15],
                                        self.vgg16_bn.layers[16],
                                        self.vgg16_bn.layers[17]
                                        ])

        self.upconv5_block = keras.Sequential(
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv4_block = keras.Sequential(
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv3_block = keras.Sequential(
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(128, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv2_block = keras.Sequential(
                                        layers.Conv2D(128, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(64, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv1_block = keras.Sequential(
                                        layers.Conv2D(64, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(config.class_num, (3, 3), padding='same'),
                                        )
        self.convlstm = ConvLSTM(input_size=(4,8),
                                 input_dim=512,
                                 hidden_dim=[512, 512],
                                 kernel_size=(3,3),
                                 num_layers=2,
                                 batch_first=False,
                                 bias=True,
                                 return_all_layers=False)
    def __call__(self, x):
        x = tf.unstack(x, axis=1)
        data = []
        for item in x:
            f1, idx1 = self.index_MaxPool(self.conv1_block(item))
            f2, idx2 = self.index_MaxPool(self.conv2_block(f1))
            f3, idx3 = self.index_MaxPool(self.conv3_block(f2))
            f4, idx4 = self.index_MaxPool(self.conv4_block(f3))
            f5, idx5 = self.index_MaxPool(self.conv5_block(f4))
            data.append(tf.expand_dims(f5, axis = 0))
        data = layers.concatenate(data, axis=0)
        lstm, _ = self.convlstm(data)
        test = lstm[0][-1,:,:,:,:]
        up6 = self.index_UnPool(test,idx5)
        up5 = self.index_UnPool(self.upconv5_block(up6), idx4)
        up4 = self.index_UnPool(self.upconv4_block(up5), idx3)
        up3 = self.index_UnPool(self.upconv3_block(up4), idx2)
        up2 = self.index_UnPool(self.upconv2_block(up3), idx1)
        up1 = self.upconv1_block(up2)
        return tf.nn.log_softmax(up1, axis=1)


class SegNet(tf.Module):
    def __init__(self):
        super(SegNet,self).__init__()
        self.vgg16_bn = tf.keras.applications.VGG16(include_top=False)
        self.relu = layers.ReLU(),
        self.index_MaxPool = layers.MaxPooling2D(pool_size=(2,2), strides=2, data_format = 'channels_last')
        self.index_UnPool = unpool(kernel_size=2, stride=2)
        # net struct

        self.conv1_block = keras.Sequential([self.vgg16_bn.layers[0],  # input layer
                                        self.vgg16_bn.layers[1],  # conv2d(3,64,(3,3))
                                        self.vgg16_bn.layers[2]  # conv2d(3,64,(3,3))
                                        ])
        self.conv2_block = keras.Sequential([self.vgg16_bn.layers[4],
                                        self.vgg16_bn.layers[5]
                                        ])
        self.conv3_block = keras.Sequential([self.vgg16_bn.layers[7],
                                        self.vgg16_bn.layers[8],
                                        self.vgg16_bn.layers[9]
                                        ])
        self.conv4_block = keras.Sequential([self.vgg16_bn.layers[11],
                                        self.vgg16_bn.layers[12],
                                        self.vgg16_bn.layers[13]
                                        ])
        self.conv5_block = keras.Sequential([self.vgg16_bn.layers[15],
                                        self.vgg16_bn.layers[16],
                                        self.vgg16_bn.layers[17]
                                        ])

        self.upconv5_block = keras.Sequential(
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv4_block = keras.Sequential(
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(512, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv3_block = keras.Sequential(
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(256, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(128, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv2_block = keras.Sequential(
                                        layers.Conv2D(128, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(64, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu
                                        )
        self.upconv1_block = keras.Sequential(
                                        layers.Conv2D(64, (3, 3), padding='same'),
                                        layers.BatchNormalization(epsilon=1e-05, momentum=0.9),
                                        self.relu,
                                        layers.Conv2D(config.class_num, (3, 3), padding='same'),
                                        )
    def __call__(self, x):
        f1, idx1 = self.index_MaxPool(self.conv1_block(x))
        f2, idx2 = self.index_MaxPool(self.conv2_block(f1))
        f3, idx3 = self.index_MaxPool(self.conv3_block(f2))
        f4, idx4 = self.index_MaxPool(self.conv4_block(f3))
        f5, idx5 = self.index_MaxPool(self.conv5_block(f4))
        up6 = self.index_UnPool(f5,idx5)
        up5 = self.index_UnPool(self.upconv5_block(up6), idx4)
        up4 = self.index_UnPool(self.upconv4_block(up5), idx3)
        up3 = self.index_UnPool(self.upconv3_block(up4), idx2)
        up2 = self.index_UnPool(self.upconv2_block(up3), idx1)
        up1 = self.upconv1_block(up2)

        return tf.nn.log_softmax(up1, axis=1)

In [17]:
model1 = up(3,3)

In [18]:
y = model1(ndata, nlabel)
print(ndata.shape)
print(nlabel.shape)
print(y.shape)

(1, 128, 256, 3)
(1, 128, 256, 1)
(1, 256, 512, 3)


In [20]:
model2 = UNet(3,3)

In [21]:
z = model2(ndata)

In [22]:
print(z.shape)

(1, 128, 256, 3)


In [26]:
model3 = ConvLSTM((128, 256), 3, 3, (3, 3), 5)

In [27]:
data = []
ndata1 = layers.concatenate([ndata, ndata], axis = 0)
for i in range(5):
    ndata1 = layers.concatenate([ndata1, ndata1], axis = 0)
ndata2 = ndata1.numpy()
for i in range(5):
    data.append(ndata2)
# print(data)
data = tf.constant(data)
print(data.shape)

(5, 64, 128, 256, 3)


In [28]:
lstm, _ = model3(data)

ResourceExhaustedError: OOM when allocating tensor with shape[64,130,258,12] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
tf.zeros([1,3,3,3])

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import numpy as np

# class double_conv(tf.Module): 
#     def __init__(self, in_ch, out_ch):
#         super(double_conv, self).__init__()
#         self.conv = keras.Sequential([
#             layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), 
#             layers.BatchNormalization(momentum=0.9),
#             layers.Activation('relu'),
#             layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), 
#             layers.BatchNormalization(momentum=0.9),
#             layers.Activation('relu')            
#         ])
#     def __call__(self, x):
#         x = self.conv(x)
#         return x

# class inconv(tf.Module):
#     def __init__(self, in_ch, out_ch):
#         super(inconv, self).__init__()
#         self.conv = double_conv(in_ch, out_ch)
#     def __call__(self, x):
#         x = self.conv(x)
#         return x

# class down(tf.Module):
#     def __init__(self, in_ch, out_ch):
#         super(down, self).__init__()
#         self.mpconv = keras.Sequential([
#             layers.MaxPooling2D(2),
#             layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), 
#             layers.BatchNormalization(momentum=0.9),
#             layers.Activation('relu'),
#             layers.Conv2D(out_ch, (3, 3), padding = "SAME", activation=None), 
#             layers.BatchNormalization(momentum=0.9),
#             layers.Activation('relu') 
#         ])

#     def __call__(self, x):
#         x = self.mpconv(x)
#         return x
        

# class up(tf.Module):
#     def __init__(self, in_ch, out_ch, bilinear=True):
#         super(up, self).__init__()

#         #  would be a nice idea if the upsampling could be learned too,
#         #  but my machine do not have enough memory to handle all those weights
#         if bilinear:
#             self.up = layers.UpSampling2D(size = (2,2), interpolation='bilinear')
#         else:
#             self.up = layers.Conv2DTranspose(in_ch//2, 2, stride=(2, 2))

#         self.conv = double_conv(in_ch, out_ch)

#     def __call__(self, x1, x2):
#         x1 = self.up(x1) 
#         diffX = x1.shape[1] - x2.shape[1] # height ?
#         diffY = x1.shape[2] - x2.shape[2] # width ?
#         padding = [[0,0], [(diffX)// 2, int((diffX + 1)/ 2)], [diffY // 2, int((diffY+1) / 2)], [0,0]]
#         x2 = tf.pad(x2, padding)
#         x = layers.concatenate([x2, x1], axis=3)
#         x = self.conv(x)
#         return x

# class outconv(tf.Module):
#     def __init__(self, in_ch, out_ch):
#         super(outconv, self).__init__()
#         self.conv = layers.Conv2D(out_ch, (1, 1), padding = "SAME", activation=None)
#     def __call__(self, x):
#         x = self.conv(x)
#         return x


# class ConvLSTMCell(tf.Module):

#     def __init__(self, input_size, input_dim, hidden_dim, kernel_size, bias):
#         """
#         Initialize ConvLSTM cell.

#         Parameters
#         ----------
#         input_size: (int, int)
#             Height and width of input tensor as (height, width).
#         input_dim: int
#             Number of channels of input tensor.
#         hidden_dim: int
#             Number of channels of hidden state.
#         kernel_size: (int, int)
#             Size of the convolutional kernel.
#         bias: bool
#             Whether or not to add the bias.
#         """

#         super(ConvLSTMCell, self).__init__()

#         self.height, self.width = input_size
#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim

#         self.kernel_size = kernel_size
#         self.padding = kernel_size[0] // 2, kernel_size[1] // 2
#         self.bias = bias


#         self.conv = keras.Sequential([
#             layers.ZeroPadding2D(padding=self.padding),
#             layers.Conv2D( #in_channels=self.input_dim + self.hidden_dim,
#                               filters=4 * self.hidden_dim,
#                               kernel_size=self.kernel_size,
#                               padding="SAME",
#                               use_bias=self.bias)
#         ])

#     def __call__(self, input_tensor, cur_state):
#         h_cur, c_cur = cur_state

#         combined = layers.concatenate([input_tensor, h_cur], axis=3)  # concatenate along channel axis

#         combined_conv = self.conv(combined)
#         for_split_list = []
        
#         for i in range(combined_conv.shape[3] // self.hidden_dim):
#             for_split_list.append(self.hidden_dim)
#         cc_i, cc_f, cc_o, cc_g = tf.split(combined_conv, for_split_list, axis=3) # split 
#         i = tf.sigmoid(cc_i)
#         f = tf.sigmoid(cc_f)
#         o = tf.sigmoid(cc_o)
#         g = tf.tanh(cc_g)

#         c_next = f * c_cur + i * g
#         h_next = o * tf.tanh(c_next)

#         return h_next, c_next

#     def init_hidden(self, batch_size):
#         return (tf.zeros([batch_size, self.height, self.width, self.hidden_dim]),
#                 tf.zeros([batch_size, self.height, self.width, self.hidden_dim]))


# class ConvLSTM(tf.Module):

#     def __init__(self, input_size, input_dim, hidden_dim, kernel_size, num_layers,
#                  batch_first=False, bias=True, return_all_layers=False):
#         super(ConvLSTM, self).__init__()

#         self._check_kernel_size_consistency(kernel_size)

#         # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
#         kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
#         hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
#         if not len(kernel_size) == len(hidden_dim) == num_layers:
#             raise ValueError('Inconsistent list length.')

#         self.height, self.width = input_size

#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim
#         self.kernel_size = kernel_size
#         self.num_layers = num_layers
#         self.batch_first = batch_first
#         self.bias = bias
#         self.return_all_layers = return_all_layers

#         cell_list = []
#         for i in range(0, self.num_layers):
#             cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

#             cell_list.append(ConvLSTMCell(input_size=(self.height, self.width),
#                                           input_dim=cur_input_dim,
#                                           hidden_dim=self.hidden_dim[i],
#                                           kernel_size=self.kernel_size[i],
#                                           bias=self.bias))

#         # self.cell_list = nn.ModuleList(cell_list)
#         self.cell_list = cell_list

In [ ]:
# cell_list = []
# height, width = 128, 256
# hidden_dim = [ 3,3,3,3,3,3,3,3,3,3,3,3]
# kernel_size = [[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3],[3,3]]
# bias = True
# for i in range(0, 10):
#     cur_input_dim = 3 

#     cell_list.append(ConvLSTMCell(input_size=(height, width),
#                                     input_dim=cur_input_dim,
#                                     hidden_dim=hidden_dim[i],
#                                     kernel_size=kernel_size[i],
#                                     bias=bias))

# # self.cell_list = nn.ModuleList(cell_list)
# cell_list = cell_list

In [ ]:
# cell_list[0].init_hidden(3)

In [30]:
tf.debugging.set_log_device_placement(True)

# 텐서 생성
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [32]:
tf.debugging.set_log_device_placement(True)

# 텐서를 CPU에 할당
with tf.device('/CPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

c = tf.matmul(a, b)
print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [38]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus[0])
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
